### Leslie Matrix

&nbsp;

The Leslie matrix is a square matrix to model and analyze the dynamics of age-structured populations. It is a projection matrix to study the growth and stability of populations over time.

In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pyodbc
os.chdir('C:/Users/tm/Downloads/utas/WildlifeDatabases')

In [2]:
#number of columns and rows in leslie matrix
maxage=5

### cleanse

In [3]:
#get data from access db
fundamentals=pd.DataFrame(columns=['Microchip','Sex','YOB'])
observations=pd.DataFrame(columns=['Microchip','TrappingDate','NumberActiveTeats'])

for i in [
         './woodbridge sandfly/Channel_database_devil_2022_06_KJS.accdb',
        ]:

    conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+f'{i};')
    fundamentals=pd.concat([fundamentals,pd.read_sql('select [Microchip],[Sex],[YOB] from fundamentals',conn)])
    observations=pd.concat([observations,pd.read_sql(
        'select [Microchip],[TrappingDate],[NumberActiveTeats] from observations',conn)]) 
    
    fundamentals.reset_index(inplace=True,drop=True)
    observations.reset_index(inplace=True,drop=True)

C:\Users\tm\AppData\Local\Temp\ipykernel_4304\1800492046.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fundamentals=pd.concat([fundamentals,pd.read_sql('select [Microchip],[Sex],[YOB] from fundamentals',conn)])
C:\Users\tm\AppData\Local\Temp\ipykernel_4304\1800492046.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  observations=pd.concat([observations,pd.read_sql(


In [4]:
#datetime col
observations['TrappingDate']=pd.to_datetime(observations['TrappingDate'])

#cleanse problematic input
observations['NumberActiveTeats']=observations['NumberActiveTeats'].str.replace('N/A','0')
observations['NumberActiveTeats'][observations['NumberActiveTeats']=='']=None
observations['NumberActiveTeats']=observations['NumberActiveTeats'].astype(float)

#remove duplicates
observations=observations.loc[observations[['Microchip','TrappingDate']].drop_duplicates().index]

C:\Users\tm\AppData\Local\Temp\ipykernel_4304\2126836536.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observations['NumberActiveTeats'][observations['NumberActiveTeats']=='']=None


In [5]:
#datetime col
fundamentals['YOB']=pd.to_datetime(fundamentals['YOB'])

#remove duplicates
fundamentals=fundamentals.loc[fundamentals['Microchip'].drop_duplicates().index]

#remove na
fundamentals=fundamentals.dropna()

In [6]:
#merge
grande=observations.merge(fundamentals,on='Microchip',how='left')

#cleanse col names
bereplaced={'Microchip':'INDIVIDUAL',
'TrappingDate':'OBSERVATION_DATE',
'YOB':'BIRTH_DATE',
'NumberActiveTeats':'NUMBER_ACTIVE_TEATS',
'Sex':'GENDER'}
grande.columns=[bereplaced[i] for i in grande.columns]

In [7]:
#sort by date
grande=grande.sort_values(['INDIVIDUAL','OBSERVATION_DATE'])

#female parent only
grande=grande[grande['GENDER']=='Female']

#devils have maximum 4 teats
grande=grande[grande['NUMBER_ACTIVE_TEATS']<=4]

#female offspring only
grande['NUMBER_ACTIVE_TEATS']/=2
grande.reset_index(inplace=True,drop=True)

#datetimeindex
grande['OBSERVATION_DATE']=pd.to_datetime(grande['OBSERVATION_DATE'],format='mixed')
grande['BIRTH_DATE']=pd.to_datetime(grande['BIRTH_DATE'])

In [8]:
#eliminate wrong birth date
grande=grande[grande['OBSERVATION_DATE']>grande['BIRTH_DATE']]

#eliminate null id
grande=grande.loc[grande['INDIVIDUAL'].dropna().index]

### estimate transition rate

In [9]:
#compute age
grande['age']=(grande['OBSERVATION_DATE']-grande['BIRTH_DATE']).apply(lambda x:x.days//365)

In [10]:
#for each year,each devil only counts once
grande['year']=grande['OBSERVATION_DATE'].dt.year
agestructure=grande.loc[grande[['year','INDIVIDUAL']].drop_duplicates().index]

In [11]:
#count
agesummary=agestructure.groupby('age').count()[['INDIVIDUAL']]
agesummary.reset_index(inplace=True)

In [12]:
#remove devil older than 10 yr which should be error
agesummary=agesummary[agesummary['age']<=10]

#move anything older than maxage yr into yr maxage category
lastrow=agesummary['INDIVIDUAL'][agesummary['age']>=maxage].sum()
agesummary['INDIVIDUAL'][agesummary['age']==maxage]=lastrow
agesummary=agesummary.iloc[:maxage+1]

In [13]:
#estimate relationship between age and frequency
#plus one to avoid log(0)
agesummary['logage']=np.log(agesummary['age']+1)

m=sm.OLS(agesummary['INDIVIDUAL'],sm.add_constant(agesummary['logage'])).fit()

In [14]:
#use fitted data to compute survivalship
agesummary['smoothed data']=m.predict()
agesummary['survivalship']=agesummary['smoothed data']/agesummary['smoothed data'].iloc[0]

### estimate fecundity

In [15]:
#for each year,each devil only counts once
fecundity=grande.groupby(['year','INDIVIDUAL']).max()[['NUMBER_ACTIVE_TEATS','age','GENDER']]
fecundity.reset_index(inplace=True)

In [16]:
#remove na female
female=fecundity[fecundity['GENDER']=='Female'].copy()
female=female.loc[female['NUMBER_ACTIVE_TEATS'].dropna().index]

#set na male to zero
male=fecundity[fecundity['GENDER']=='Male'].copy()
fecundity=pd.concat([female,male])
fecundity['NUMBER_ACTIVE_TEATS']=fecundity['NUMBER_ACTIVE_TEATS'].fillna(0)

In [17]:
#comput mean fecundity
reproduction=fecundity[['NUMBER_ACTIVE_TEATS','age']].groupby('age').mean()
reproduction.reset_index(inplace=True)

In [18]:
#remove devil older than 10 yr which should be error
reproduction=reproduction[reproduction['age']<=10]

#move anything older than maxage yr into yr maxage category
lastrow=reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']>=maxage].mean()
reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']==maxage]=lastrow
reproduction=reproduction.iloc[:maxage+1]

C:\Users\tm\AppData\Local\Temp\ipykernel_4304\3778900559.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']==maxage]=lastrow


### compute leslie matrix

In [19]:
#generate leslie matrix
lesliematrix=np.zeros((maxage+1,maxage+1))
lesliematrix[0]=reproduction['NUMBER_ACTIVE_TEATS'].tolist()
for i in range(1,maxage+1):
    lesliematrix[i][i-1]=agesummary['survivalship'].iloc[i]

In [20]:
#show no disease matrix
for i in lesliematrix:
    print(i)

[0.         0.70098039 1.48630137 1.76785714 1.57142857 0.25      ]
[0.70943304 0.         0.         0.         0.         0.        ]
[0.         0.53946227 0.         0.         0.         0.        ]
[0.         0.         0.41886608 0.         0.         0.        ]
[0.         0.         0.         0.32532441 0.         0.        ]
[0.         0.         0.         0.         0.24889531 0.        ]


In [21]:
#dominant eigenvalue still smaller than one
#the population eventually will decline
max(np.abs(np.linalg.eigvals(lesliematrix)))

1.1313860085388119

In [22]:
lesliematrix

array([[0.        , 0.70098039, 1.48630137, 1.76785714, 1.57142857,
        0.25      ],
       [0.70943304, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.53946227, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.41886608, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.32532441, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.24889531,
        0.        ]])

In [23]:
lesliematrix.flatten().tolist()

[0.0,
 0.7009803921568627,
 1.4863013698630136,
 1.7678571428571428,
 1.5714285714285714,
 0.25,
 0.7094330411487038,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.539462266272108,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4188660822974074,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.32532441479719504,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.24889530742081187,
 0.0]